In [ ]:
import pandas as pd

In [ ]:
df_rating = pd.read_csv("u.data.csv", sep = '\t', names=["user id", "item id", "rating", "timestamp"])
df_rating.head()

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
df_rating.describe()

,user id,item id,rating,timestamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [ ]:
# Исходя из практики 1 части выгружаем только информационные столбцы. Не берем: video release date, IMDb URL, unknown 
df_movies = pd.read_csv("u.item.csv", sep = '|', usecols = [0,1,2,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23], names=["movie id", "movie title", "release date", "Action", "Adventure", "Animation", "Children's",
                                    "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"],encoding = "ISO-8859-1");
df_movies.head()


,movie id,movie title,release date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
lc = df_movies.columns.tolist()
lc[15]

'Mystery'

In [ ]:
# Находим ползователя, поставившего больше всего оценок
df_rating.groupby(["user id"])['rating'].agg(['count']).sort_values(by='count', ascending = False).head(10)

,count
user id,
405,737
655,685
13,636
450,540
276,518
416,493
537,490
303,484
234,480


In [ ]:
# Формируем фрейм фильмов, которые оценил user id 405
df_rating_user = df_rating.loc[df_rating['user id'] == 405]
df_rating_user.head(15)

,user id,item id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447
12472,405,1409,1,885549045
12484,405,953,3,885546487
12604,405,994,1,885549746
12607,405,387,1,885546680
12718,405,1432,1,885549942


In [ ]:
# Формируем фрейм фильмов по количеству оценок и суммарной оценке
df_rating_agg = df_rating.groupby(["item id"])['rating'].agg(['count', 'sum'])
df_rating_agg.head(10)  

,count,sum
item id,,
1,452,1753
2,131,420
3,90,273
4,209,742
5,86,284
6,26,93
7,392,1489
8,219,875
9,299,1165


In [ ]:
# Преобразуем release date в формат Date и выделяем из даты год
import datetime as dt
df_movies["release date"]=pd.to_datetime(df_movies["release date"])
df_movies["release date"]= df_movies["release date"].dt.year
df_movies=df_movies.fillna(0)
df_movies["release date"]=df_movies['release date'].astype('int')
df_movies.head(5)

,movie id,movie title,release date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1995,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),1995,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),1995,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
# Формируем фрейм для обучения
df_merge = df_rating_user.merge(df_rating_agg, on = 'item id', how ='inner')
df_merge = df_merge.merge(df_movies, left_on = 'item id', right_on = 'movie id', how = 'inner')
df_merge = df_merge[['item id', 'rating', 'count', 'sum', 'release date', "Action", "Adventure", "Animation", "Children's",
                     "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]]
df_merge.head(5)

,item id,rating,count,sum,release date,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,56,4,394,1600,1994,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,592,1,9,30,1995,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,1582,1,1,1,1947,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,171,1,65,252,1991,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,580,1,32,108,1995,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
df_merge.describe()

,item id,rating,count,sum,release date,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
count,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,...,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000
mean,732.933514,1.834464,71.725916,261.242877,1987.953867,0.165536,0.099050,0.040706,0.104478,0.301221,...,0.016282,0.016282,0.081411,0.032564,0.028494,0.149254,0.073270,0.134328,0.052917,0.028494
std,470.160223,1.345183,84.519925,340.297551,12.841659,0.371916,0.298932,0.197741,0.306087,0.459100,...,0.126645,0.126645,0.273651,0.177614,0.166492,0.356580,0.260756,0.341236,0.224020,0.166492
min,2.000000,1.000000,1.000000,1.000000,1922.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,381.000000,1.000000,11.000000,31.000000,1988.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,668.000000,1.000000,40.000000,129.000000,1994.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1110.000000,2.000000,101.000000,351.000000,1995.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1592.000000,5.000000,583.000000,2541.000000,1998.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# Выделяем признаки для модели обучения, настраиваем выборки
from sklearn.model_selection import train_test_split
X, y = df_merge[['count', 'sum', 'release date', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', "Crime", "Documentary", "Drama", "Fantasy", 
           "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]], df_merge['rating'];
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


In [ ]:
# Создать и обучить модель LinearRegression
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
# Оцениваем качество модели
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
# по учебной выборке
np.sqrt(mean_squared_error(y_train, model.predict(X_train)))

1.0971457875146846

In [ ]:
# по тестовой выборке
np.sqrt(mean_squared_error(y_test, model.predict(X_test)))

1.0594978350018223